In [9]:
import requests
import urllib.request
import lxml.html as lh
import pandas as pd
import

In [10]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = requests.get(url)

In [11]:
response

<Response [200]>

In [12]:
doc = lh.fromstring(response.content)

In [13]:
tr_elements = doc.xpath('//tr')

In [14]:
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [15]:
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print('%d:"%s"'%(i,name))
    col.append((name,[]))

1:"Postcode"
2:"Borough"
3:"Neighbourhood
"


In [16]:
for c in range(1,len(tr_elements)):
    #T is our c'th row
    T=tr_elements[c]
    
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [17]:
col

[('Postcode',
  ['M1A',
   'M2A',
   'M3A',
   'M4A',
   'M5A',
   'M6A',
   'M6A',
   'M7A',
   'M8A',
   'M9A',
   'M1B',
   'M1B',
   'M2B',
   'M3B',
   'M4B',
   'M4B',
   'M5B',
   'M5B',
   'M6B',
   'M7B',
   'M8B',
   'M9B',
   'M9B',
   'M9B',
   'M9B',
   'M9B',
   'M1C',
   'M1C',
   'M1C',
   'M2C',
   'M3C',
   'M3C',
   'M4C',
   'M5C',
   'M6C',
   'M7C',
   'M8C',
   'M9C',
   'M9C',
   'M9C',
   'M9C',
   'M1E',
   'M1E',
   'M1E',
   'M2E',
   'M3E',
   'M4E',
   'M5E',
   'M6E',
   'M7E',
   'M8E',
   'M9E',
   'M1G',
   'M2G',
   'M3G',
   'M4G',
   'M5G',
   'M6G',
   'M7G',
   'M8G',
   'M9G',
   'M1H',
   'M2H',
   'M3H',
   'M3H',
   'M3H',
   'M4H',
   'M5H',
   'M5H',
   'M5H',
   'M6H',
   'M6H',
   'M7H',
   'M8H',
   'M9H',
   'M1J',
   'M2J',
   'M2J',
   'M2J',
   'M3J',
   'M3J',
   'M4J',
   'M5J',
   'M5J',
   'M5J',
   'M6J',
   'M6J',
   'M7J',
   'M8J',
   'M9J',
   'M1K',
   'M1K',
   'M1K',
   'M2K',
   'M3K',
   'M3K',
   'M4K',
   'M4K',
   'M5

#Checking for the length of the table.  Ensuring all columns are the same.

In [18]:
[len(C) for (title,C) in col]

[287, 287, 287]

#Creating a dictionary and dataframe from the table scrapped from wiki.

In [19]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [20]:
df.head()

,Postcode,Borough,Neighbourhood\n
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


#Dropping extra Boroughs that are not assigned

In [21]:
df = df[df.Borough != 'Not assigned']

In [22]:
df

,Postcode,Borough,Neighbourhood\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n
5,M6A,North York,Lawrence Heights\n
6,M6A,North York,Lawrence Manor\n
...,...,...,...
281,M8Z,Etobicoke,Kingsway Park South West\n
282,M8Z,Etobicoke,Mimico NW\n
283,M8Z,Etobicoke,The Queensway West\n
284,M8Z,Etobicoke,Royal York South West\n


#Reset the index since we lost a few rows.

In [23]:
df.reset_index()

,index,Postcode,Borough,Neighbourhood\n
0,2,M3A,North York,Parkwoods\n
1,3,M4A,North York,Victoria Village\n
2,4,M5A,Downtown Toronto,Harbourfront\n
3,5,M6A,North York,Lawrence Heights\n
4,6,M6A,North York,Lawrence Manor\n
...,...,...,...,...
205,281,M8Z,Etobicoke,Kingsway Park South West\n
206,282,M8Z,Etobicoke,Mimico NW\n
207,283,M8Z,Etobicoke,The Queensway West\n
208,284,M8Z,Etobicoke,Royal York South West\n


#Checking the neighborhood values for odd variables

In [24]:
df['Neighbourhood\n'].values

array(['Parkwoods\n', 'Victoria Village\n', 'Harbourfront\n',
       'Lawrence Heights\n', 'Lawrence Manor\n', "Queen's Park\n",
       'Not assigned\n', 'Rouge\n', 'Malvern\n', 'Don Mills North\n',
       'Woodbine Gardens\n', 'Parkview Hill\n', 'Ryerson\n',
       'Garden District\n', 'Glencairn\n', 'Cloverdale\n', 'Islington\n',
       'Martin Grove\n', 'Princess Gardens\n', 'West Deane Park\n',
       'Highland Creek\n', 'Rouge Hill\n', 'Port Union\n',
       'Flemingdon Park\n', 'Don Mills South\n', 'Woodbine Heights\n',
       'St. James Town\n', 'Humewood-Cedarvale\n', 'Bloordale Gardens\n',
       'Eringate\n', 'Markland Wood\n', 'Old Burnhamthorpe\n',
       'Guildwood\n', 'Morningside\n', 'West Hill\n', 'The Beaches\n',
       'Berczy Park\n', 'Caledonia-Fairbanks\n', 'Woburn\n', 'Leaside\n',
       'Central Bay Street\n', 'Christie\n', 'Cedarbrae\n',
       'Hillcrest Village\n', 'Bathurst Manor\n', 'Downsview North\n',
       'Wilson Heights\n', 'Thorncliffe Park\n', 'Adela

In [25]:
df

,Postcode,Borough,Neighbourhood\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n
5,M6A,North York,Lawrence Heights\n
6,M6A,North York,Lawrence Manor\n
...,...,...,...
281,M8Z,Etobicoke,Kingsway Park South West\n
282,M8Z,Etobicoke,Mimico NW\n
283,M8Z,Etobicoke,The Queensway West\n
284,M8Z,Etobicoke,Royal York South West\n


#Dropping the \n character

In [26]:
df['Neighbourhood\n'] = df['Neighbourhood\n'].map(lambda x: x.rstrip('\n'))
df.rename(columns={"Neighbourhood\n": "Neighbourhood"}, inplace=True)

#Replacing the neighborhood value with the Borough if the neighborhood is not assigned

In [27]:
df.Neighbourhood\n.replace("Not assigned", df.Borough, inplace=True)

SyntaxError: unexpected character after line continuation character (<ipython-input-27-1e5d5b6547a3>, line 1)

In [31]:
df

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
...,...,...,...
281,M8Z,Etobicoke,Kingsway Park South West
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West


Combine the neighborhoods together based on the Postal Code

In [33]:
groupb_table = df.groupby(['Postcode','Borough'], sort=False).agg( ', '.join)
df=groupb_table.reset_index()
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park


#Previewing new frame to check for accuracy

In [35]:
df.shape

(103, 3)

<H1>Question 2

#Had to rename the column to match spatial data and pull necessary coordinates

In [38]:
df.rename(columns={"Postcode": "Postal Code"}, inplace=True)

In [39]:
geospatial=pd.read_csv('http://cocl.us/Geospatial_data')

In [40]:
geospatial

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


<h2> Combine the two tables and make sure it looks good

In [41]:
df2 = pd.merge(df, geospatial, on='Postal Code')

In [42]:
df2

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558
101,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So...",43.636258,-79.498509


<h1> Question 3

In [46]:
import geopy
import folium